# 学習曲線

## 精度・汎化性能を向上させる手段<a name="method"></a>

- 訓練データを増やす
- 正則化項を強める
- パラメータ数を減らしてモデルを単純化する

## 選択基準<a name="criteria"></a>

学習曲線を表示すると、どの手段を選択すれば良いかの指針になる。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

titles = ['High bias', 'High variance', 'Balanced']
labels = ['Expected', 'Training', 'Validation']

x_min, x_max = 1, 10
x = np.linspace(x_min, x_max, 50)
expected = .8
train = np.array([
    [
        np.exp(- x * .9) * 1.5 + expected / 2 + .01,
        np.exp(- x * .3) * .25 + expected + .01,
        np.exp(- x * .6) * .35 + expected + .01
    ], [
        - np.exp(- x * .6) * 1.1 + .59,
        - np.exp(- x * .6) * .35 + .19,
        - np.exp(- x * .6) * .35 + .19
    ]
])
validation = np.array([
    [
        - np.exp(- x * .9) + expected / 2 - .01,
        - np.exp(- x * .2) * .95 + expected - .01,
        - np.exp(- x * .6) * 1.5 + expected - .01
    ], [
        np.exp(- x * .6) * .75 + .61,
        np.exp(- x * .2) * .95+ .21,
        np.exp(- x * .6) * 1.45 + .21
    ]
])

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for row, cols in enumerate(axes):
    for col, ax in enumerate(cols):
        ax.plot(x, train[row, col], color='g', label=labels[1])
        ax.plot(x, validation[row, col], color='r', label=labels[2])

        ax.set_xlim(x_min, x_max)
        ax.set_ylim(0, 1)
        ax.set_xticklabels(())

        if row == 0:
            ax.set_title(titles[col])
            ax.plot([x_min, x_max], [expected, expected], color='b', label=labels[0])
            
            if col == 0:
                ax.legend()
                ax.set_ylabel('Accuracy')
        
        else:
            ax.set_xlabel('Training sample')
            ax.set_xticklabels([1] + [''] * 8 + ['n'])
            
            if col == 0:
                ax.set_ylabel('Loss')
        
        if row + col > 0:
            ax.set_yticklabels(())

plt.show()

- 訓練データでの精度が求める水準に達していない・損失が十分に低下しない
 - = 未学習・高バイアス
   - → パラメータの数を増やす（特徴を追加）・正則化を弱める
- 訓練データでは求める精度を達成している・損失は十分に低下している
 - バリデーションデータと訓練データで精度・損失が大きく乖離している
   - = 過学習・高バリアンス
     - → さらにデータを集める・正則化を強める(正則化の適用できないモデルでは次元削減などで特徴を減らす)
 - バリデーションデータと訓練データの精度・損失が近い
   - = 最適なモデル

### 学習曲線の描画<a name="learning_curve"></a>

#### データの準備<a name="data"></a>

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

loader = load_digits()
X, y = loader.data, loader.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

#### 学習曲線表示<a name="visualization"></a>

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression(random_state=0, n_jobs=-1)
train_sizes, train_scores, test_scores = learning_curve(estimator=estimator, X=X_train, y=y_train,
                                                        train_sizes=np.linspace(.1, 1., 50), cv=10, n_jobs=-1)
train_score_mean = train_scores.mean(axis=1)
test_score_mean = test_scores.mean(axis=1)

plt.plot(train_sizes, train_score_mean, color='g', label='Train')
plt.plot(train_sizes, test_score_mean, color='r', label='Validation')

plt.legend(loc='lower right')
plt.xlabel('Train sample')
plt.ylabel('Accuracy')
plt.xlim(train_sizes.min(), train_sizes.max())
plt.ylim(0.9, 1.01)

plt.show()